In [23]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd
import re

from os.path import basename

In [24]:
#path_ref = '../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL'
#path_ref_name = 'Thumb-Arm-Atmega-Leon3-RiscV-RECIPE-DECIMAL-'
path_ref = '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES'
path_ref_name = 'Thumb-Arm-Atmega-Leon3-RiscV-RECIPE-DECIMAL-'

In [25]:
# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()
for root, dirs, files in os.walk(path_ref):
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)
            
nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

pd.options.display.max_colwidth = 100
print('nameFileS:'+nameFileS)
print('pathFileS:'+pathFileS)
print('rootFileS:'+rootFileS)

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

0     nameFileS:parameters.json
1     nameFileS:parameters.json
2     nameFileS:parameters.json
3     nameFileS:parameters.json
4     nameFileS:parameters.json
5     nameFileS:parameters.json
6     nameFileS:parameters.json
7     nameFileS:parameters.json
8     nameFileS:parameters.json
9     nameFileS:parameters.json
10    nameFileS:parameters.json
dtype: object
0     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/l...
1     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/m...
2     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/f...
3     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/c...
4     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/b...
5     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/i..

In [26]:
nameFileTim = [] # Timing
pathFileTim = [] # Timing
rootFileTim = [] # Timing

# path = os.getcwd()
for root, dirs, files in os.walk(path_ref):
    for file in files:
        if file.endswith("CC4CSFrameworkExecutionTime.csv"):
            nameFileTim.append(file)
            pathFileTim.append(os.path.join(root, file))
            rootFileTim.append(root)


In [27]:
pathFileTim

['../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/lud/Leon3/double/optO5-Ofast/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/lud/Leon3/double/optO2-09/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/lud/Leon3/double/optO3-05/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/lud/Leon3/double/optO6-Og/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/lud/Leon3/double/optO4-Os/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/lud/Leon3/double/optO2-05/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_

In [28]:
# Timing file
# Creating Empty DataFrame and Storing it in variable df
dfTimTemp = pd.DataFrame()
dfTimTot = pd.DataFrame()

for i in range(0, len(pathFileTim)):
    path = pathFileTim[i]
    try:
        dfTimTempNew = pd.read_csv(path, sep=',',engine='python', header=None)
    except:
        print(path)
        
    if os.name == 'nt':
        spR = rootFileTim[i].split('\\')
    else:
        spR = rootFileTim[i].split('/')
        
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfTimTemp['DEVICE'] = device
    dfTimTemp['BOARD'] = board
    dfTimTemp['FUNCTION'] = function
    dfTimTemp['DATA_TYPE'] = datatype
    dfTimTemp['OPT_FLAG'] = optFlag
    dfTimTemp['MV_SIZE'] = matrixVectorSize
    
    # dfTimTemp['inputName'] = dfTimTemp['inputName'].map(lambda x: x.lstrip('values_'))
    # dfTimTemp['inputName'] = dfTimTemp['inputName'].astype(int)
    # dfTimTemp.sort_values('inputName')
    
    # Filter the dataframe to get rows where the first column matches the search string
    filtered_df = dfTimTempNew[dfTimTempNew[0] == 'InputGenerationTime']
    dfTimTemp['InputGenerationTime'] = filtered_df[1].tolist()
    
    filtered_df = dfTimTempNew[dfTimTempNew[0] == 'ProfilingTime']
    dfTimTemp['ProfilingTime'] = filtered_df[1].tolist()
    
    filtered_df = dfTimTempNew[dfTimTempNew[0] == 'SimulationTime']
    dfTimTemp['SimulationTime'] = filtered_df[1].tolist()

    filtered_df = dfTimTempNew[dfTimTempNew[0] == 'FramaCExecutionTime']
    dfTimTemp['FramaCExecutionTime'] = filtered_df[1].tolist()
    
    filtered_df = dfTimTempNew[dfTimTempNew[0] == 'TotalExecutionTime']
    dfTimTemp['TotalExecutionTime'] = filtered_df[1].tolist()
    
    if i == 0:
        dfTimTot = dfTimTemp
    else:
        frames = [dfTimTot, dfTimTemp]
        dfTimTot = pd.concat(frames)


In [29]:
dfTimTot

,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,InputGenerationTime,ProfilingTime,SimulationTime,FramaCExecutionTime,TotalExecutionTime
0,Leon3,Leon3,lud,double,optO2-09,32x32,0.021843,11.529591,7.381105,49.553066,894.242105
0,Leon3,Leon3,lud,double,optO2-09,32x32,0.021843,11.529591,7.381105,49.553066,894.242105
0,Leon3,Leon3,lud,double,optO3-05,32x32,0.021843,13.531958,7.556321,49.553066,1035.104981
0,Leon3,Leon3,lud,double,optO6-Og,32x32,0.021843,11.818465,7.073961,49.553066,1099.779321
0,Leon3,Leon3,lud,double,optO4-Os,32x32,0.021843,14.144073,9.354174,49.553066,1058.648352
...,...,...,...,...,...,...,...,...,...,...,...
0,RiscV,RiscV,shell_sort,float,optO2-08,32x32,0.025346,9.877018,47.360031,35.518347,778.991376
0,RiscV,RiscV,shell_sort,float,optO1-01,32x32,0.025346,9.777072,47.521913,35.518347,150.483167
0,RiscV,RiscV,shell_sort,float,optO3-02,32x32,0.025346,10.080002,47.605437,35.518347,1008.011819
0,RiscV,RiscV,shell_sort,float,optO1-00,32x32,0.025346,9.888808,47.722726,35.518347,93.162779


In [30]:
dfTimTot.to_csv("TotalTiming.csv", sep=';', index=False, encoding='utf-8')

In [14]:
import pandas as pd

# Paths to the CSV files
csv1_path = 'dataset/xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-1.csv'
csv2_path = 'dataset/xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-2.csv'

# Load the CSV files into DataFrames
df1 = pd.read_csv(csv1_path)
df2 = pd.read_csv(csv2_path, sep=';',engine='python', header=0, names=df1.columns)

# Concatenate the DataFrames along the rows (axis=0)
concatenated_df = pd.concat([df1, df2], ignore_index=True)

# Path to save the concatenated CSV file
merged_csv_path = 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-MERGED.csv'

# Save the concatenated DataFrame to a new CSV file
concatenated_df.to_csv(merged_csv_path, index=False, encoding='utf-8')

print(f'Concatenated DataFrame saved to: {merged_csv_path}')

Concatenated DataFrame saved to: xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-MERGED.csv
